In [38]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphs
using Catlab.Graphs.BasicGraphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Graphics.GraphvizGraphs
using Catlab.Graphs.PropertyGraphs
using Catlab.CategoricalAlgebra.CSets
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures: SortedSet

In [39]:
draw(g) = to_graphviz(g, node_labels=true)

draw (generic function with 1 method)

In [40]:
struct Preorder
    carrier:: Set{String}
    relation :: SortedSet{Pair{String,String}}
    """Construct valid preorder by taking reflexive/transitive closure"""
    function Preorder(carrier:: Set{String}, rel :: Vector{Pair{String,String}})
      for (a,b) ∈ rel
        a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
      end
      relation = SortedSet(rel)
      for (a, b, c) in Iterators.product(carrier, carrier, carrier)
        if ((a => b) ∈ relation) && ((b => c) ∈ relation)
          push!(relation, a => c) # enforces relation is transitive
        end
      end
      for i in carrier
        push!(relation, i => i) # enforces that relation is reflexive
      end
      return new(carrier, relation)
    end
  end

In [41]:
struct Monotone_map
    domain::Preorder
    codomain::Preorder
    mapping::FinFunction
    function Monotone_map(domain::Preorder, cod::Preorder, mapping::FinFunction)
      ((dom(mapping) == domain.carrier) && (codom(mapping) == cod.carrier)
      ) || error("mapping mismatch")
      return new(domain,cod,mapping)
    end
  end

In [42]:
function is_monotone(mm::Monotone_map)::Bool
    for relation in mm.domain.relation
      p = relation[1]
      q = relation[2]
      if !((mm.mapping(p) => mm.mapping(q)) in mm.codomain.relation)
        return false
      end
    end
    return true
  end

is_monotone (generic function with 1 method)

In [43]:
function load_major(path::String)::Preorder
    file = open(path, "r")
    classes = Set{String}()
    relations = Vector{Pair{String, String}}()
    while !eof(file)
        line = readline(file)
        if length(strip(line, ' ')) != 0
            sp = split(line, ":")

            if length(sp) >= 1
                name = strip(sp[1], ' ')
                push!(classes, string(name))
            end

            if length(sp) >= 2
                reqs_str = strip(sp[2], ['[', ']', ' '])
                reqs = split(reqs_str, ",")
                
                for req in reqs
                    req = strip(req, ' ')
                    if length(req) > 0
                        if !in(req, classes)
                            push!(classes, req)
                        end
                        push!(relations, req => name)
                    end
                end
            end
        end
    end
    close(file)
    preorder = Preorder(classes, relations)
    return preorder
end

load_major (generic function with 1 method)

In [44]:
cs = load_major("reqs/CS.reqs")

Preorder(Set(["MAC2312", "CIS4914", "ENC3246", "PHY2049L", "CNT4007", "COP3530", "MAC2313", "COP4020", "CIS4301", "COT3100"  …  "COP3503", "CEN3031", "PHY2048", "MAS3114", "STA3032", "COP3502", "CDA3101", "PHY2048L", "MAC2311", "PHY2049"]), SortedSet(["CDA3101" => "CDA3101", "CDA3101" => "CIS4301", "CDA3101" => "CNT4007", "CDA3101" => "COP4600", "CEN3031" => "CEN3031", "CIS4301" => "CIS4301", "CIS4914" => "CIS4914", "CNT4007" => "CNT4007", "COP3502" => "CDA3101", "COP3502" => "CEN3031"  …  "PHY2048" => "PHY2048", "PHY2048" => "PHY2048L", "PHY2048" => "PHY2049", "PHY2048L" => "PHY2048", "PHY2048L" => "PHY2048L", "PHY2048L" => "PHY2049", "PHY2049" => "PHY2049", "PHY2049L" => "PHY2049", "PHY2049L" => "PHY2049L", "STA3032" => "STA3032"],
Base.Order.ForwardOrdering()))

In [45]:
cs.carrier

Set{String} with 23 elements:
  "MAC2312"
  "CIS4914"
  "ENC3246"
  "PHY2049L"
  "CNT4007"
  "COP3530"
  "MAC2313"
  "COP4020"
  "CIS4301"
  "COT3100"
  "COP4600"
  "EGS4034"
  "COP4533"
  "COP3503"
  "CEN3031"
  "PHY2048"
  "MAS3114"
  "STA3032"
  "COP3502"
  "CDA3101"
  "PHY2048L"
  "MAC2311"
  "PHY2049"

In [46]:
cs.relation

SortedSet{Pair{String, String}, Base.Order.ForwardOrdering} with 92 elements:
  "CDA3101" => "CDA3101"
  "CDA3101" => "CIS4301"
  "CDA3101" => "CNT4007"
  "CDA3101" => "COP4600"
  "CEN3031" => "CEN3031"
  "CIS4301" => "CIS4301"
  "CIS4914" => "CIS4914"
  "CNT4007" => "CNT4007"
  "COP3502" => "CDA3101"
  "COP3502" => "CEN3031"
  "COP3502" => "CIS4301"
  "COP3502" => "COP3502"
  "COP3502" => "COP3503"
  "COP3502" => "COP3530"
  "COP3502" => "COP4020"
  "COP3502" => "COP4533"
  "COP3502" => "COP4600"
  "COP3502" => "COT3100"
  "COP3503" => "CDA3101"
  "COP3503" => "CEN3031"
  "COP3503" => "CIS4301"
  "COP3503" => "CNT4007"
  "COP3503" => "COP3503"
  "COP3503" => "COP3530"
  "COP3503" => "COP4020"
  ⋮ 